In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data Visulization/Data Preprocessing

In [ ]:
train=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")


In [ ]:
train.info()

In [ ]:
sample=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv")


In [ ]:
test=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv")
test.head()

In [ ]:
train.head()

In [ ]:
train.drop(["date_block_num"],axis=1,inplace=True)

date_block_num is not an important parameter for us,we are gonna drop that

In [ ]:
train["date"]=pd.to_datetime(train["date"],infer_datetime_format=True)

To approach any time series problem our first aim is to convert Date from an object dtype to Datetime dtype

In [ ]:
train=train.set_index(['date'])

Second Aim is to remove all the index and set Date column as our index variable

In [ ]:
train=train[['item_cnt_day','shop_id', 'item_id', 'item_price']]

We r shuffling column(it gonna help us later)

In [ ]:
train.head()

In [ ]:
from matplotlib.pylab import rcParams
rcParams['figure.figsize']=10,6

In [ ]:
train["item_cnt_day"].plot()
plt.xlabel("item_cnt_day")

In [ ]:
train.columns

In [ ]:
tr=train

In [ ]:
j=1
a=['item_cnt_day', 'shop_id', 'item_id', 'item_price']
for i in a:
    tr[i+str(j)]=tr[i].shift(-1)
    

This code is generating  4 exta column (suppose our column contain entry shown below as {old column} after going through above loop the newly column generated will contain the values starting fron 2nd index of old column

Old column -------------------New column

0--------------------------1

1---------------------------2

2---------------------------NaN

In [ ]:
tr

In [ ]:
tr.drop(["item_cnt_day","shop_id1","item_id1","item_price1"],axis=1,inplace=True)

We r gonna drop unneccasry column{item_cnt_day1 is gonna used as aour label}

In [ ]:
tr

In [ ]:
tr.fillna(method='ffill',inplace=True)

In [ ]:
tr

Now we r gonna separate data into label and features and then apply MinMaxScaler

In [ ]:
yy=tr["item_cnt_day1"].values

In [ ]:
yy = yy.reshape((len(yy), 1))

In [ ]:
tr.drop(["item_cnt_day1"],axis=1,inplace=True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scale=MinMaxScaler()
scale=scale.fit(tr)
w=scale.transform(tr)

In [ ]:
scaler=scale.fit(yy)
qq=scaler.transform(yy)

Remember above scaler we gonaa used later

In [ ]:
qq

In [ ]:
www=w

# LSTM Model

In [ ]:
xtrain,xtest,ytrain,ytest=www[:2348680,0:3],www[2348680:,0:3],qq[:2348680,-1],qq[2348680:,-1]

We cant use Train/Test Split in time series problem because all the label have the value dependent on prev value

    For Eg.-suppose we have to separate this data into train and test{1,2,3,4,5,6},,if we apply train test then our train data can contain{2,6,1,4} and test {5,3}

But a time Series problem requires data to be in form train={1,2,3,4} test={5,6

In [ ]:
xtest

In [ ]:
ytest

In [ ]:
xtrain = xtrain.reshape(xtrain.shape[0], xtrain.shape[1], 1)
xtest = xtest.reshape(xtest.shape[0], xtest.shape[1], 1)

LSTM model requires 3D data as input so we our converting train features and test features in 3D

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM,Dropout
from keras.layers import Bidirectional

In [ ]:
model=Sequential()
model.add(LSTM(100,activation="relu",return_sequences=True,input_shape=(xtrain.shape[1],xtrain.shape[2])))
model.add(Dropout(0.1))
model.add(LSTM(100,activation="relu",return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(100,activation="relu",return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(100,activation="relu",return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(50,activation="relu"))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.fit(xtrain,ytrain,validation_data=(xtest,ytest),epochs=50,batch_size=30000,verbose=2)

In [ ]:
ypred=model.predict(xtest)

In [ ]:
ypred[:5]

In [ ]:
ypred=scaler.inverse_transform(ypred)

we our using above scaler to transfrom to denormalize ypred

In [ ]:
ypred

In [ ]:
ytest = ytest.reshape((len(ytest), 1))

Converting 1d data to 2d

In [ ]:
y_test = scaler.inverse_transform(ytest)

In [ ]:
y_test

In [ ]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_test,ypred))

In [ ]:
y_test[:5]

In [ ]:
ypred[:5]

Prediction item_price for Test data

In [ ]:
test

No to implement above model to predict item_cnt_day we need item_price in our test data,so we r gonna predict item_price first than gonna used above model to find item_cnt_day

To predict item_price all off the above process will be repeat{we will shift the features,do minmaxscaler,gonna build LSTM model again,than we will predict item_price

In [ ]:
train

In [ ]:
train=train[['item_price','shop_id', 'item_id']]

In [ ]:
trr=train

In [ ]:
j=1
a=['item_price','shop_id', 'item_id']
for i in a:
    trr[i+str(j)]=trr[i].shift(-1)
    

In [ ]:
trr

In [ ]:
trr.drop(["shop_id1","item_id1","item_price"],axis=1,inplace=True)

In [ ]:
trr.fillna(method='ffill',inplace=True)

In [ ]:
yyy=trr["item_price1"].values
trr.drop(["item_price1"],axis=1,inplace=True)

In [ ]:
yyy=yyy.reshape(len(yyy),1)

In [ ]:
yyy

In [ ]:
trr

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scale=MinMaxScaler()
scale=scale.fit(trr)
w=scale.transform(trr)

In [ ]:
www=w

In [ ]:
yyy

In [ ]:
scalerr=scale.fit(yyy)
wewe=scalerr.transform(yyy)

Note here scalerr is used as transforming function for our variable

In [ ]:
xtrain,xtest,ytrain,ytest=www[:2348680,0:2],www[2348680:,0:2],wewe[:2348680,-1],wewe[2348680:,-1]

In [ ]:
xtes=xtest

In [ ]:
xtrain = xtrain.reshape(xtrain.shape[0], xtrain.shape[1], 1)
xtest = xtest.reshape(xtest.shape[0], xtest.shape[1], 1)

In [ ]:
modelt=Sequential()
modelt.add(LSTM(100,activation="relu",return_sequences=True,input_shape=(xtrain.shape[1],xtrain.shape[2])))
modelt.add(Dropout(0.1))
modelt.add(LSTM(100,activation="relu",return_sequences=True))
modelt.add(Dropout(0.1))
modelt.add(LSTM(100,activation="relu",return_sequences=True))
modelt.add(Dropout(0.1))
modelt.add(LSTM(100,activation="relu",return_sequences=True))
modelt.add(Dropout(0.1))
modelt.add(LSTM(50,activation="relu"))
modelt.add(Dense(1))
modelt.compile(optimizer='adam', loss='mse')

In [ ]:
modelt.fit(xtrain,ytrain,validation_data=(xtest,ytest),epochs=50,batch_size=30000,verbose=2)

In [ ]:
test.head()

In [ ]:
test.drop(["ID"],axis=1,inplace=True)

In [ ]:
te=test

In [ ]:
scalll=scale.fit(test.values)
w=scalll.transform(test.values)

In [ ]:
w

In [ ]:
w=w.reshape(w.shape[0], w.shape[1], 1)

Point to notice that we our feeding our test data to our item predicting model so we need to convert that into 3D Because our LSTM model only needs 3d Input

In [ ]:
y_pred=modelt.predict(w)

In [ ]:
y_pred

In [ ]:
scalerr=scale.fit(yyy)
wewe=scalerr.transform(yyy)

Remember the above scalerr it gonna used as transfoming function(if have just repeated the above code to avaid any confusion)

In [ ]:
y_pred=scalerr.inverse_transform(y_pred)

In [ ]:
y_pred

In [ ]:
test

In [ ]:
len(y_pred)

In [ ]:
test["wewe"]=y_pred

finally "wewe" feature contain all of our item_price value for out test data

In [ ]:
test

# Final Data Transformation

In [ ]:
scalert=scale.fit(test.values)
wewe=scalert.transform(test.values)

here wewe contains all our 2d normalize test values

In [ ]:
wewe

In [ ]:
y_prrr=wewe.reshape(wewe.shape[0], wewe.shape[1], 1)

In [ ]:
y_predict=model.predict(y_prrr)

Note here "model" is tha model which we build first

In [ ]:
y_predict

In [ ]:
scaler=scale.fit(yy)
qq=scaler.transform(yy)

The above code is that scaler which firstly transform the label(i have repeated that code again to avoid any confusion)

In [ ]:
y_predict=scaler.inverse_transform(y_predict)

In [ ]:
y_predict

Hurrah!we have got our final output in y-predict,now we will convert it into dataframe and set it for submission

In [ ]:
submission=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv")

In [ ]:
submission

In [ ]:
pred=y_predict.flatten()

In [ ]:
for i in range(214200):
    pred[i]=round(pred[i])

In [ ]:
pred

In [ ]:
r = pd.Series(pred,name="item_cnt_month")

In [ ]:
submiss = pd.concat([pd.Series(range(0,214200),name = "ID"),r],axis = 1)

In [ ]:
submiss

In [ ]:
submiss.to_csv("11.csv",index=False)